In [ ]:
#import libraries
import requests
import pandas as pd
from datetime import datetime
import os

In [ ]:
#download dataset
df_lines = pd.read_csv('./Online_Data/referentiel-des-lignes.csv', sep=';')
stops_data = pd.read_csv('./Online_Data/arrets.csv', sep=';')
delays_metro = pd.read_csv("./Collected_Data/metro_delays.csv")
delays_rer = pd.read_csv("./Collected_Data/rail_delays.csv")
metro_incident = pd.read_csv("./Collected_Data/metro_line_reports.csv")
rer_incident = pd.read_csv("./Collected_Data/rer_line_reports.csv")
trafic2023_ratio = pd.read_csv("./Online_Data/validations-1er-semestre.csv", sep=';')
trafic2023_raw = pd.read_csv("./Online_Data/validations-reseau.csv", sep=";")


In [ ]:
#clean the dataset
line_refs = df_lines[(~df_lines['TransportSubmode'].isin(['suburbanRailway', 'regionalRail', 'railShuttle']))]
line_refs = line_refs[['ID_Line', 'TransportMode', 'Name_Line']]
line_refs = line_refs.sort_values(by='ID_Line')

stops_data = stops_data[stops_data['ArRType'].isin(['metro', 'rail'])]
stops_data = stops_data.sort_values(by=['ArRType', 'ArRId'])
stops_data = stops_data[['ArRId', 'ArRName', 'ArRType', 'ArRTown']]

metro_incident['ref'] = metro_incident['ref'].str.replace('stop_point:IDFM:', '', regex=False)
metro_incident['ref'] = metro_incident['ref'].str.replace('line:IDFM:', '', regex=False)
metro_incident['ref'] = metro_incident['ref'].str.replace('stop_area:IDFM:', '', regex=False)

rer_incident['ref'] = rer_incident['ref'].str.replace('stop_point:IDFM:', '', regex=False)
rer_incident['ref'] = rer_incident['ref'].str.replace('line:IDFM:', '', regex=False)
rer_incident['ref'] = rer_incident['ref'].str.replace('stop_area:IDFM:', '', regex=False)

delays_metro.drop(['scheduled_arrival','scheduled_departure','arrival_difference','departure_difference'], axis=1, inplace=True) #inplace allows to actually modify the original datset without having to reassing with "=""

In [ ]:
#create filter
stops_filter = [22086, 463013, 22136, 462993, 21964, 462969, 22125, 463113, 41295, 473921, 473993, 41354, 474060, 474061]
name_filter = ["CH.D.G.ETOILE", "CHATELET", "SAINT-LAZARE","ST-GERM.D.PRES", "BLANCHE","AVENUE DU PRESIDENT KENNEDY","BUNO GIRONVILLE","MASSY PALAISEAU"]
lines_filter = ["C01371", "C01372", "C01374" ," C01382", "C01742", "C01743", "C01727", "C0172"]

In [ ]:
#filter dataset
stops_data = stops_data[stops_data['ArRId'].isin(stops_filter)]

metro_incident = metro_incident[
    metro_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

rer_incident = rer_incident[
    rer_incident['ref'].isin(map(str, stops_filter + lines_filter))
]

trafic2023_ratio = trafic2023_ratio[trafic2023_ratio['LIBELLE_ARRET'].isin(name_filter)]

trafic2023_raw = trafic2023_raw[trafic2023_raw['LIBELLE_ARRET'].isin(name_filter)]

In [ ]:
#merge dataset
merged_metro = pd.merge(delays_metro, stops_data, left_on='stop_reference', right_on='ArRId')
merged_metro.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode'], axis=1, inplace=True) 

merged_rer = pd.merge(delays_rer, stops_data, left_on='stop_reference', right_on='ArRId')
merged_rer.drop(['ArRId', 'ArRName', 'ArRType', 'transport_mode'], axis=1, inplace=True)